In [1]:
import pandas as pd
from transformers import pipeline
import os 
import numpy as np
from tqdm import tqdm
import datasets
import ast
from dotenv import load_dotenv
from huggingface_hub import login

In [2]:
# log into huggingface with token to acces llama + gemma models
load_dotenv()  # reads .env
hf_token = os.getenv("HF_TOKEN")
login()

In [3]:
# read data
data_path = os.path.join('..', 'data', "monopolet_cleaned.csv")
df = pd.read_csv(data_path)

# Convert the column back to list
df['acceptable_solutions_cleaned'] = df['acceptable_solutions_cleaned'].apply(ast.literal_eval)

In [4]:
# prompt:

good_system_prompt = """
Du er deltager i et panel, der diskuterer personlige dilemmaer. 
Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. 
Det er MEGET vigtigt, at du kun skal gøre dette med maks 120 tokens. 
Du skal heller ikke bare give mig et enkelt ord som svar. 
Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. 
Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."""

In [5]:
def set_prompts(system_prompt, dilemma):
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_prompt
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Her er dilemmaet: {dilemma}. Råd: "
                }
            ],
        },
    ]
    return messages

### Gemma-3B

In [8]:
def set_prompts(system_prompt, dilemma):
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_prompt
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"Her er dilemmaet: {dilemma}. Råd: "
                }
            ],
        },
    ]
    return messages

In [7]:
# loading model from huggingface Hub
from transformers import AutoProcessor, AutoModelForMultimodalLM

processor = AutoProcessor.from_pretrained("google/gemma-3-12b-it")
model = AutoModelForMultimodalLM.from_pretrained("google/gemma-3-12b-it")

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/1065 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [13]:
messages = set_prompts(llama_prompt, df['first_person_dilemma'].iloc[1789])
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=80)
print(processor.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Tal med din svigerinde igen. Respekter hendes ønske om ikke at blande dig direkte i hendes forhold, men tilbyd din støtte og lyt uden at dømme. Opmuntre hende til at søge professionel rådgivning for at tackle den økonomiske ubalance og dens indvirkning på hendes trivsel.<end_of_turn>


In [14]:
df['acceptable_solutions'].iloc[1789]

["Tag en støttende, fortrolig snak med svigerinden alene: Per skal sætte sig ned med sin svigerinde, lytte til hende, lade hende 'tømme sin mentale rygsæk', hjælpe hende reflektere over sine drømme og grænser, og lade hende beslutte, om hun vil blive i forholdet (respektér hvis hun siger 'du må ikke sige noget'). (Deltagere: Per og svigerinden.)",
 'Hjælp svigerinden med at få professionel støtte: foreslå og hjælp hende med at finde en psykolog eller anden fagperson, så hun kan få lagt sit liv og økonomiske følelser på bordet og få klarhed, før der træffes beslutninger om forholdet. (Rækkefølge: først samtale/afklaring med svigerinden, herefter opsøge professionel hjælp. Deltagere: Per, svigerinden og evt. psykolog.)',
 "Forsigtigt spørge ind til parrets trivsel uden at moralisere over økonomien: Per kan nænsomt spørge enten svigerinden eller begge parter hvordan de har det, om de er glade, og komme med en lille 'heads up' om, at han oplever en ubalance — men gøre det på en omsorgsfuld

### Test Mistral model

In [62]:
from vllm import LLM
from vllm.sampling_params import SamplingParams
from datetime import datetime, timedelta

#SYSTEM_PROMPT = ""

#user_prompt = "Give me 5 non-formal ways to say 'See you later' in French."

#messages = [
    #{
        #"role": "system",
        #"content": SYSTEM_PROMPT
    #},
    #{
        #"role": "user",
        #"content": user_prompt
    #},
#]

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
llm = LLM(model=model_name, tokenizer_mode="mistral")

sampling_params = SamplingParams(max_tokens=60, temperature=0.15)
outputs = llm.chat(messages, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)
# Here are five non-formal ways to say "See you later" in French:

# 1. **À plus tard** - Until later
# 2. **À toute** - See you soon (informal)
# 3. **Salut** - Bye (can also mean hi)
# 4. **À plus** - See you later (informal)
# 5. **Ciao** - Bye (informal, borrowed from Italian)

# ```
#  /\_/\
# ( o.o )
#  > ^ <
# ```


ModuleNotFoundError: No module named 'vllm'

### Llama-3.1-8B-Instruct

In [23]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token # we need to specify the pad token specifically
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [15]:
llama_prompt = """
Du er deltager i et panel, der diskuterer personlige dilemmaer. 
Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. 
Det er MEGET vigtigt, at du kun skal gøre dette med maks 120 tokens. 
Du skal heller ikke bare give mig et enkelt ord som svar. 
Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. 
Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."""

In [ ]:
messages = set_prompts(llama_prompt, df['first_person_dilemma'].iloc[456])

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

print(messages)
outputs = model.generate(
	**inputs, 
	max_new_tokens=200,
	temperature=0.3,
	do_sample=True
	)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': [{'type': 'text', 'text': "\nDu er deltager i et panel, der diskuterer personlige dilemmaer. \nDu skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. \nDet er MEGET vigtigt, at du kun skal gøre dette med maks 200 tokens. \nDu skal heller ikke bare give mig et enkelt ord som svar. \nDu skal ikke give mig svaret i jeg-person, men blot komme med et råd. \nDu skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Her er dilemmaet: Mit navn er Mathias. Jeg har løjet for min kone Molly om, hvad jeg laver efter arbejde. Til daglig har jeg et fysisk krævende job og har fri kl. 14. På det tidspunkt er jeg så træt og drænet, at jeg hver dag kører til en lille, afsidesliggende rasteplads for at tage en morfar på cirka en time. Luren giver mig fornyet energi, og bagefter kører jeg hjem, så jeg rammer hoveddøren kl. 16. Molly tror, jeg har fri kl. 15.30, og hun

In [35]:
# Loop over rows 30 to 40 (inclusive of 30, exclusive of 41)
for i in tqdm(range(30, 36)):
    # Create messages for the current row
    messages = set_prompts(llama_prompt, df['first_person_dilemma'].iloc[i])
    
    # Prepare the input for the model
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)
    
    # Print the messages for reference
    #print(f"Row {i} messages:\n", messages)
    
    # Generate model outputs
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.3,
        do_sample=True
    )
    
    # Decode and print the output
    decoded_output = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:])
    print(f"Row {i} output:\n", decoded_output)
    print("="*50)  # Separator for readability


 17%|█▋        | 1/6 [01:13<06:08, 73.72s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Row 30 output:
 Sæt en timer på enhederne, så de er automatisk på lydløs efter en vis tid. Dette kan hjælpe din mor med at huske at slå dem på lydløs, uden at du selv skal være involveret i konflikten.<|eot_id|>


 33%|███▎      | 2/6 [02:59<06:09, 92.36s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Row 31 output:
 Man skal ikke nødvendigvis unfollow de mennesker, man tidligere har været seksuelt sammen med, når man er i et forhold. Det er vigtigt at respektere hinandens grænser og holdninger, men også at være ærlig og åben omkring ens handlinger og følelser. I dette tilfælde kan det være en god ide at have en åben og respektfuld


 50%|█████     | 3/6 [04:14<04:14, 84.79s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Row 33 output:
 Vær ærlig, men præsentér dig selv som en kvinde med en klar vision for fremtiden. Fortæl om dine styrker, dine interesser og dine mål. Fokus på, hvad du kan tilbyde, og hvordan du kan bidrage til en andens liv.<|eot_id|>


 83%|████████▎ | 5/6 [07:04<01:25, 86.00s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Row 34 output:
 Du skal ikke ændre dit efternavn tilbage til dit pigenavn, selv om din eksmand foreslår det. Det er din egen beslutning, og du har allerede sagt nej til det under skilsmissen. Din eksmands nye giftemål skal ikke påvirke din beslutning. Du skal prioritere dine egne følelser og behov, og ikke lade andre bestemme over din identitet.<|eot_id|>


100%|██████████| 6/6 [08:30<00:00, 85.02s/it]

Row 35 output:
 Indfør en begrænsning på antallet af gaver, der gives til børnebørn, f.eks. én gave til jul og én fødselsdagsgave om året. Dette vil hjælpe med at reducere udgifterne og gøre det lettere at håndtere gaveræset.<|eot_id|>


### Evaluation

In [178]:
evaluate_prompt = """Du er en hjælpsom assistent, der skal hjælpe med at bedømme en række udsagn. 
Giv en score mellem 0-1 på, hvor ens disse to råd er. Du skal give mig et decimaltal.
Du kan kun bedømme det på INDHOLDET, og ikke længden, sproget eller andet. 
Så du skal konkret vurderer, om rådet er det samme i de to eksempler.
En score tættere på 0 indikerer, at de rådgiver om helt forskellige ting. 
En score tættere på 1 indikerer, at de rådgiver om de samme ting.
Du skal kun outputte scoren."""

In [179]:
score_prompt = """Her er rådene, du skal bedømme:
 1. Bed din mor om at sætte sin iPhone/iPad på lydløs, når I er sammen, så I kan have en samtale uden baggrundsstøj.
 2. Smid din mors telefon ud og sig hun er dum"""

In [180]:

    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": evaluate_prompt
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": score_prompt
                }
            ],
        },
    ]

In [181]:
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

print(messages)
outputs = model.generate(
	**inputs, 
	max_new_tokens=200,
	temperature=0.3,
	do_sample=True
	)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

NameError: name 'tokenizer' is not defined

In [ ]:
llama_prompt

"\nDu er deltager i et panel, der diskuterer personlige dilemmaer. \nDu skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. \nDet er MEGET vigtigt, at du kun skal gøre dette med maks 120 tokens. \nDu skal heller ikke bare give mig et enkelt ord som svar. \nDu skal ikke give mig svaret i jeg-person, men blot komme med et råd. \nDu skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."

In [ ]:
df['acceptable_solutions'].iloc[30]

['Tag en snak med din mor igen og sig klart, at du gerne vil have hendes fulde opmærksomhed — især når notifikationerne begynder at forstyrre jeres samvær; bed hende lytte til dig.',
 'Bed din mor om at sætte sin iPhone/iPad på lydløs, når I er sammen (både hjemme hos hende og ude), så I kan have en samtale uden baggrundsstøj.',
 'Vis hende, hvordan man slår notifikationer fra (deaktiverer app‑lyde) og hjælp hende med at slå dem fra, når hun skal være social, og slå dem til igen senere — det kan laves som en hyggelig/legende øvelse mellem jer.',
 "Giv hende venligt 'sociale spilleregler' og basic etiquette for sociale medier (takt og tone): forklar, at man ikke underretter hele selskabet hele tiden, og at der er passende måder at være online på i sociale sammenhænge."]

In [ ]:
df['first_person_dilemma'].iloc[456]

'Mit navn er Mathias. Jeg har løjet for min kone Molly om, hvad jeg laver efter arbejde. Til daglig har jeg et fysisk krævende job og har fri kl. 14. På det tidspunkt er jeg så træt og drænet, at jeg hver dag kører til en lille, afsidesliggende rasteplads for at tage en morfar på cirka en time. Luren giver mig fornyet energi, og bagefter kører jeg hjem, så jeg rammer hoveddøren kl. 16. Molly tror, jeg har fri kl. 15.30, og hun roser mig ofte for, hvor frisk og energisk jeg er, når jeg kommer hjem. Jeg støvsuger, laver mad, vasker tøj og leger med børnene, og det er den times søvn, der gør det muligt. Molly er hjemme det meste af dagen, fordi hun skriver speciale, og hun har nok at se til med at hente og aflevere børnene hver dag — noget jeg teknisk set godt kunne hjælpe mere med, hvis jeg kom tidligere hjem. Jeg ved dog bare, at hvis jeg dropper min daglige morfar, vil jeg sandsynligvis ikke kunne bidrage på samme måde, som jeg gør nu. Derfor fortsætter jeg med den lille hvide løgn, se

In [ ]:
df['acceptable_solutions'].iloc[456]

['Fortæl Molly sandheden: sig, at du har løjet om dine eftermiddagslure, forklar at du gør det fordi du har brug for en morfar for at kunne fungere, og forhandl åbent videre — tilbud hende et tilsvarende privilegium (så hun også kan få en time for sig selv) og vær villig til at justere hvor ofte luren tages (fx kun 3 dage om ugen eller andet aftalt antal dage).',
 "Sig det på rette tidspunkt og i en god timing: 'jeg skal fortælle dig noget — hver dag imellem X og Y sover jeg, så jeg kan være til stede for jer' — få det ud af systemet, så I kan tale om praktiske løsninger sammen.",
 'Sæt dig ned med dig selv og undersøg, hvorfor du begyndte at lyve (fx konfliktskyhed); betro dig til din ægtefælle om dit behov, og arbejd sammen om at finde måder I begge kan passe på jer selv og hinanden i småbørnsperioden.']

### Testing batch processing + batch prompting

In [6]:
prompts = []
for i, row in df.iterrows():
    prompt = set_prompts(good_system_prompt, row['first_person_dilemma'])
    prompts.append(prompt)

df['prompts'] = prompts

In [7]:
# convert df to huggingface dataset for batch processing
ds = datasets.Dataset.from_pandas(df)

In [8]:
import datasets
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
# specify tokenizer & model

# loading model from huggingface Hub
from transformers import AutoProcessor, AutoModelForMultimodalLM

tokenizer = AutoProcessor.from_pretrained("google/gemma-3-12b-it")
model = AutoModelForMultimodalLM.from_pretrained("google/gemma-3-12b-it")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading weights:   0%|          | 0/1065 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoProcessor

# load your tokenizer
#tokenizer = AutoProcessor.from_pretrained("google/gemma-3-12b-it")

# Example dataframe
# df = pd.DataFrame({'acceptable_solutions': [['Hello world', 'This is a test'], ['Another example']]})

# Flatten all strings in the column
all_strings = [s for lst in df['acceptable_solutions_cleaned'] for s in lst]

# Tokenize each string and count tokens
token_counts = [len(tokenizer(s).input_ids) for s in all_strings]

# Convert to numpy array for easy stats
token_counts = np.array(token_counts)

# Summary statistics for the entire column (per string)
print("Number of strings:", len(token_counts))
print("Mean tokens per string:", token_counts.mean())
print("Min tokens per string:", token_counts.min())
print("Max tokens per string:", token_counts.max())


In [9]:
ds_subset = ds.select(range(15))

In [10]:
# print example of prompt
messages = ds_subset[0]["prompts"]

prompt_text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)
print(prompt_text)

<bos><start_of_turn>user

Du er deltager i et panel, der diskuterer personlige dilemmaer. 
Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. 
Det er MEGET vigtigt, at du kun skal gøre dette med maks 120 tokens. 
Du skal heller ikke bare give mig et enkelt ord som svar. 
Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. 
Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet.

Her er dilemmaet: Mit navn er Mette, og vores veninde Amalie skal giftes til foråret, så hun skal selvfølgelig have en polterabend. Jeg taler på vegne af alle os, der skal deltage, for vi er havnet i et dilemma. Amalie har givet udtryk over for sin mor, at hun rigtig gerne vil vide, hvornår polterabend'en finder sted. Hun har ondt i maven over, at hun ikke kender datoen, og hun har desuden sagt til sin kommende mand, at hun bliver vred, hvis han ikke fortæller hende datoen. Amalie er ikke så slem, som det lyder — hun er bare lidt af en kon

In [25]:
tokenizer_text = tokenizer.tokenizer  # unwrap the text tokenizer

token_counts = []

for lst in df['acceptable_solutions_cleaned']:
    for s in lst:
        tokens = tokenizer_text.tokenize(s)
        token_counts.append(len(tokens))


AttributeError: TokenizersBackend has no attribute tokenizer

In [23]:
f"Mean: {np.mean(token_counts)}, min: {np.min(token_counts)}, max: {np.max(token_counts)}"

'Mean: 44.804086753898545, min: 2, max: 137'

In [26]:
# test same with llama tokenizer;

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

token_counts = []

for lst in df['acceptable_solutions_cleaned']:
    for s in lst:
        tokens = tokenizer.tokenize(s)
        token_counts.append(len(tokens))

f"Mean: {np.mean(token_counts)}, min: {np.min(token_counts)}, max: {np.max(token_counts)}"

'Mean: 50.670998386807675, min: 3, max: 151'

In [27]:
np.std(token_counts)

np.float64(15.277997595515806)

In [ ]:
def tokenize_chat(batch):
    chat_prompts = [
        # convert prompts into a chat template
        tokenizer.apply_chat_template( # create chat template used by the loaded model
            messages,
            add_generation_prompt=True,
            tokenize=False, # not tokenizing yet
        )
        for messages in batch["prompts"]
    ]

    # tokenize the chat_template prompts
    tokenized_batch = tokenizer(
        chat_prompts,
        padding=True, # need to add padding for gpu compatability; add padding if sentences do not have the same amount of tokens
        truncation=True,
        return_tensors=None,
    )

    return tokenized_batch

In [ ]:
tokenized_ds = ds_subset.map(
    tokenize_chat,
    batched=True,
    remove_columns=ds_subset.column_names,
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

### Test Qwen-8B

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto"
    #device_map="auto"
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [46]:
def set_prompts_qwen(system_prompt, dilemma):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Her er dilemmaet: {dilemma}. Råd: "}
    ]


In [49]:
llama_prompt

"Du er deltager i et panel, der diskuterer personlige dilemmaer. Du skal give et konkret, utvetydigt svar og kun give mig svaret på dilemmaet. Det er MEGET vigtigt, at du kun skal gøre dette med maks 50 tokens. Du skal heller ikke bare give mig et enkelt ord som svar. Du skal ikke give mig svaret i jeg-person, men blot komme med et råd. Du skal ikke starte med at skrive 'sådan håndterer du det:', men gå lige til rådet."

In [53]:
messages = set_prompts_qwen(llama_prompt, df['first_person_dilemma'].iloc[1000])

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=80
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
#try:
    # rindex finding 151668 (</think>)
 #   index = len(output_ids) - output_ids[::-1].index(151668)
#except ValueError:
 #   index = 0

#thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[0:], skip_special_tokens=True).strip("\n")

#print("thinking content:", thinking_content)
print(content)

Betale, men sige klart, at du ikke kan fortsætte med at betale, når det fordobles.


In [56]:
df['first_person_dilemma'].iloc[1000]

'Jeg skiftede for omtrent et år siden fra det private til det offentlige. I min nye afdeling har man længe haft en morgenmadsordning, hvor man på skift står for at købe ind til fælles morgenmad hver anden uge. Da der er mange i afdelingen, bliver det til mange omgange på et år, og det ender som minimum omkring 1.200 kr. årligt for mig. Derudover er der en gavekasse, som samlet set medfører en årlig udgift på omkring 2.000 kr. Jeg bakker gerne op om det sociale på arbejdspladsen og er en del af en gruppe, som frivilligt arrangerer sociale aktiviteter efter arbejdstid, men jeg mener principielt, at det ikke skal koste så mange penge at være en del af en arbejdsplads. Jeg synes, det er dårligt, at der ikke er afsat penge i afdelingsbudgettet til en månedlig morgenmad, og jeg mener også, at arbejdspladsen burde stå for at sende blomster ved barsel og ved fratrædelser. Dilemmaet er blevet mere aktuelt, fordi der er fremsat forslag om at have morgenmad hver uge, hvilket vil fordoble udgifter

In [57]:
df['acceptable_solutions'].iloc[1000]

["Stå frem offentligt og sig fra (starte en lille 'revolution') — for eksempel rejse dig op og markere over for kollegerne; gør det charmerende og humoristisk (fx Spartacus-parodi) så du ikke fremstår som pedantisk; involver: Linea og de kolleger, der vil bakke op; betingelse: gør det på en måde, der får andre med.",
 'Medbring din egen morgenmad og spis sammen med kollegerne — deltag socialt uden at betale til morgenmadsordningen; involver: Linea og kollegerne ved fælles morgenmad.',
 'Meld dig ud af ordningerne / stop med at betale eller deltage, hvis det strider imod dine principper — vælg at stå udenfor de frivillige ordninger; involver: Linea (konsekvens: mulig sladder/stigmatisering, men legitim mulighed).',
 'Fortsæt med at betale for at bevare den gode stemning — vælg at bidrage økonomisk hvis du foretrækker at undgå konflikt og holde ro i afdelingen; involver: Linea.',
 'Brug dokumentation (fx et Excel-ark) og tal konkret om økonomien — vis hvor meget du reelt har brugt om åre